In [1]:
import itertools
import functools
import string
import termcolor
import re
from collections import defaultdict, Counter
import numpy as np

## Input data

In [359]:
def read_quotes():
    with open("cytaty.txt", "r") as fstream:
        return list(map(lambda x: x.strip(), fstream.readlines()))


def read_words_base_forms():
    words_base_forms = defaultdict(lambda: [])
    with open("base_forms.txt", "r") as fstream:
        bases_words = map(lambda x: x.split(";")[0:2], fstream.readlines())
        for base, word in bases_words:
            words_base_forms[word].append(base)
    return dict(words_base_forms)

## Quotes

In [360]:
def clean_prefix_of_quote(quote):
    FILTERED_CHAR_CODES = set([8211])
    for i in range(len(quote)):
        if quote[i] == " " or quote[i] in string.punctuation:
            continue
        if ord(quote[i]) in FILTERED_CHAR_CODES:
            continue
        return quote[i:]
    return quote


def clean_quote_numbers(quote):
    regex_match = re.match("([\d]*\.[\d\s\.]*)(.*)", quote)
    if regex_match is None:
        return quote
    return regex_match.group(2)


def clean_html_suffix(quote, regex):
    regex_match = re.match(regex, quote)
    if regex_match is None:
        return quote
    return regex_match.group(1)


def fix_punctuation(quote):
    if len(quote) == 0 or quote[-1] in set(["?", "!", "."]):
        return quote
    return f"{quote}."


def clean_quote(quote):
    quote = clean_prefix_of_quote(quote)
    quote = clean_quote_numbers(quote)
    quote = clean_html_suffix(quote, regex="(.*?)&lt;(.*)")
    quote = clean_html_suffix(quote, regex="(.*?)&gt;(.*)")
    return fix_punctuation(quote)

    
def split_on_regex_subquotes(quote, regex, next_part_group):
    subquotes = []
    while True:
        match = re.match(regex, quote)
        if match is None:
            break
        subquotes.append(match.group(1))
        quote = match.group(next_part_group)
    subquotes.append(quote)
    return subquotes


def split_on_subquotes(quote):
    sentence_subquotes = split_on_regex_subquotes(quote, regex="(.*?)&lt;(.*?)&gt; ([A-Z](.*))", next_part_group=3)
    return itertools.chain(*map(
        lambda x: split_on_regex_subquotes(x, regex="(.*?)\. ([A-Z](.*))", next_part_group=2),
        sentence_subquotes
    ))


def is_quote_accepted(quote):
    MIN_QUOTE_LENGTH = 4
    return len(quote.split(" ")) >= MIN_QUOTE_LENGTH


def get_formatted_quotes(quotes):
    subquotes = itertools.chain(*map(split_on_subquotes, quotes))
    cleaned_subquotes = map(clean_quote, subquotes)
    filtered_subquotes = filter(is_quote_accepted, cleaned_subquotes)
    return list(filtered_subquotes)

## Quotes index

In [386]:
class QuotesIndex(object):
    
    def __init__(self, words_base_forms, quotes):
        self.words_base_forms = words_base_forms
        self.quotes = quotes
        self.index_of_quotes = self._generate_index_of_quotes()
        self.word_scores = self._generate_word_scores()
        
    @staticmethod
    def _clean_word(word):
        regex_match = re.match("[^a-z]*([a-z]*)[^a-z]*", word)
        return regex_match.group(1)

    @staticmethod
    def _tokenize_sentence(sentence):
        return [QuotesIndex._clean_word(x) for x in sentence.lower().split(' ')]

    def _generate_word_scores(self):
        SCORE_MULTIPLIER = 5
        word_counter = Counter()
        word_scores = {}
        for quote in self.quotes:
            base_forms = self._generate_base_forms(quote)
            for base_form in base_forms:
                word_counter[base_form] += 1
        max_word_count = max(word_counter.values())
        for word in word_counter.keys():
            word_scores[word] = np.exp(-(word_counter[word] / max_word_count) * SCORE_MULTIPLIER)
        return defaultdict(lambda: 1.0, word_scores)
        
    def _generate_base_forms(self, sentence):
        words = QuotesIndex._tokenize_sentence(sentence)
        base_forms = []
        for word in words:
            if word in self.words_base_forms:
                base_forms.extend(self.words_base_forms[word])
            else:
                base_forms.append(word)
        return set(base_forms)

    def _generate_index_of_quotes(self):
        index_of_quotes = defaultdict(lambda: set())
        for index, quote in enumerate(self.quotes):
            base_forms = self._generate_base_forms(quote)
            for base_form in base_forms:
                index_of_quotes[base_form].add(index)
        return index_of_quotes

    def _generate_document_indexes_scores(self, query):
        words = QuotesIndex._tokenize_sentence(query)
        word_base_forms = {x: self._generate_base_forms(x) for x in words}
        indexes_scores = defaultdict(lambda: 0)
        for word, base_forms in word_base_forms.items():
            base_forms_matching_indexes = {x: self.index_of_quotes[x] for x in base_forms}
            if len(base_forms_matching_indexes) == 0:
                continue
            for base_form, matching_indexes in base_forms_matching_indexes.items():                
                for matching_index in matching_indexes:
                    indexes_scores[matching_index] += self.word_scores[base_form]
        return indexes_scores
    
    def get_documents_scores(self, query):
        indexes_scores = self._generate_document_indexes_scores(query)
        return {self.quotes[index]: indexes_scores[index] for index in indexes_scores.keys()}

## Experiments

In [413]:
def generate_answer(quotes_index, query, past_answers):
    DEFAULT_QUERY = "Być albo nie być."
    documents_scores = quotes_index.get_documents_scores(query)
    for document in documents_scores.keys():
        if document in past_answers:
            del documents_scores[document]
    if len(documents_scores) == 0 and query == DEFAULT_QUERY: 
        return DEFAULT_QUERY
    if len(documents_scores) == 0:
        return generate_answer(quotes_index, query=DEFAULT_QUERY, past_answers=past_answers)
    documents, scores = zip(*documents_scores.items())
    boosted_scores = np.power(scores, 10)
    normalized_scores = boosted_scores / np.sum(boosted_scores)
    return np.random.choice(documents, p=normalized_scores)

In [ ]:
quotes = get_formatted_quotes(read_quotes())
words_base_forms = read_words_base_forms()
quotes_index = QuotesIndex(words_base_forms, quotes)

In [ ]:
past_answers = set()
while True:
    query = input()
    answer = generate_answer(quotes_index, query, past_answers)
    print(termcolor.colored(answer, "green"))

 Ładna dziś pogoda.


Bank jest miejscem, w którym pożyczają ci parasol wówczas, gdy jest ładna pogoda, i proszą o jego zwrot, gdy zaczyna padać deszcz.


 Więc poszedłeś dziś do banku?


To było wielkie świństwo wobec pracowników PGR, że nie zostali dopuszczeni do prywatyzacji – jak pracownicy banków czy innych przedsiębiorstw.


 O jakiej sytuacji mówisz?


Geniusz Bruke'a polegał na tym, że pojął przesłanki i konsekwencje takiego sposobu myślenia, jaki wyraził się w hasłach i w dokonaniach rewolucji francuskiej, reagując więc na to doświadczenie, upierał się przy twierdzeniu, że tam gdzie chodzi o sprawy człowieka, nic nie istnieje „w ogóle”, a tylko w szczegółach, abstrakcyjne zaś myślenie jest najgorszą z możliwych wskazówek postępowania.


 Widzę, że znasz się na historii francuskiej.


Historia Rasputina oraz fakt, że do jego zamordowania przyznał się książę Jusupow, są powszechnie znane.


 Ja nie słyszałem o tym fakcie.


Dyskusję o miejscu pochówku głowy państwa prowokuje dziś to samo środowisko, które sześć lat temu w najostrzejszych słowach oburzało się zadawaniem pytań, czy Czesław Miłosz powinien spocząć na krakowskiej Skałce.


In [405]:
np.power(sorted(quotes_index.get_documents_scores("Pilot jest pijany.").values(), reverse=True), 10)

array([5.51627405e+04, 5.51627405e+04, 1.58854641e+03, ...,
       1.89363829e-10, 1.89363829e-10, 1.89363829e-10])

In [415]:
quotes[0]

'Drzewo nie jest podmiotem.'